In [ ]:
import torch
import torchvision
import torch.nn as nn

In [ ]:
model = torchvision.models.resnet101()
num_iterations = 10
xe = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 5e-4)

In [ ]:

batch_size = 50   # this works, 100 does not
for i in range(num_iterations):
    inputs = torch.randn(batch_size,3,224,224)
    labels = torch.LongTensor(batch_size).random_(0, 100)
    loss = xe(model(inputs), labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Done one batch")

In [ ]:
desired_batch_size = 100
tolerable_batch_size = 50
accum_steps = desired_batch_size / tolerable_batch_size

In [ ]:
for i in range(num_iterations):
    inputs = torch.randn(tolerable_batch_size,3,224,224)
    labels = torch.LongTensor(tolerable_batch_size).random_(0, 100)
    loss = xe(model(inputs), labels)
    # The loss needs to be scaled to have the same significance over the whole dataset, because the mean should be taken across
    # the whole dataset , which requires the loss to be divided by the number of batches. This is only the case if the loss
    # returned is averaged. But if the loss returned is summed (for example: nn.BCELoss(reduction = 'sum')) then we do not need to do this
    loss = loss / accum_steps
    loss.backward()

    if ((i + 1) % accum_steps == 0) or (i + 1 == num_iterations):
        optimizer.step()
        optimizer.zero_grad()
        print("Done one batch")